In [91]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [92]:
import client.engine as engine

In [93]:
intro = engine.get_info()
intro

{'attn_module_name_format': 'model.layers.{}.self_attn',
 'layer_name_format': 'model.layers.{}',
 'max_seq_length': 2048,
 'mlp_module_name_format': 'model.layers.{}.mlp',
 'model_name': 'LLaMa-30b',
 'n_attn_head': 52,
 'n_embd': 6656,
 'n_layer': 60}

In [94]:
intro["layer_name_format"]

'model.layers.{}'

In [95]:
# ? engine.submit

In [96]:
job_id = engine.submit(
    prompt = [
        "Michael Jordan plays the sport of",
        "The Space Needle is located in the city of"
    ],
    max_new_tokens= 10,
    get_answers= True,
    top_k = 5,
    layers= [intro["layer_name_format"].format(l) for l in range(5, 10)]
)
print(job_id)

INFO: 2023-04-11 17:14:48,921 - => Submitting request...
INFO: 2023-04-11 17:14:48,928 - => Successfully submitted job '7vYS4AE3vyBpzSxqmHBSpX'
INFO: 2023-04-11 17:14:48,929 - => Dumped request for job '7vYS4AE3vyBpzSxqmHBSpX' to /disk/u/arnab/Projects/engine/jobs/7vYS4AE3vyBpzSxqmHBSpX
7vYS4AE3vyBpzSxqmHBSpX


In [97]:
# ? engine.retrieve

In [98]:
result = engine.retrieve(job_id)

INFO: 2023-04-11 17:15:06,290 - => Retrieving job '7vYS4AE3vyBpzSxqmHBSpX'...
INFO: 2023-04-11 17:15:07,875 - => Retrieved job '7vYS4AE3vyBpzSxqmHBSpX'
INFO: 2023-04-11 17:15:09,956 - => Dumped response for job '7vYS4AE3vyBpzSxqmHBSpX' to /disk/u/arnab/Projects/engine/jobs/7vYS4AE3vyBpzSxqmHBSpX


In [99]:
result["description"]

'Your job has been completed'

In [100]:
result.keys()

dict_keys(['job_id', 'status', 'timestamp', 'description', 'data'])

In [101]:
result['data'][0].keys()

dict_keys(['generated_text', 'answer', 'input_tokenized', 'generated_tokens', 'activations'])

In [102]:
result['data'][0]['activations'].keys()

dict_keys(['model.layers.5', 'model.layers.6', 'model.layers.7', 'model.layers.8', 'model.layers.9'])

In [103]:
import torch
import json
import copy

In [105]:
torch.tensor(result['data'][0]['activations']['model.layers.7']).shape

torch.Size([1, 7, 6656])

In [106]:
for r in result['data']:
    print("txt = ", r['generated_text'])
    print("input_tokenized = ", r['input_tokenized'])
    print("answer = ", r['answer'])
    print('generated_tokens = ', r['generated_tokens'])
    print("activations")
    for layer in r['activations']:
        print(f"     {layer} : {torch.tensor(r['activations'][layer]).shape}")

    print()
    

txt =  [' Michael Jordan plays the sport of basketball. Michael Jordan is a basketball player.']
input_tokenized =  [[' <s>', 1], [' Michael', 5765], [' Jordan', 18284], [' plays', 13582], [' the', 278], [' sport', 7980], [' of', 310]]
answer =  [{'top_token': ' basketball', 'candidates': [{'token': ' basketball', 'token_id': 20305, 'p': 0.4729}, {'token': ' golf', 'token_id': 29416, 'p': 0.1355}, {'token': ' baseball', 'token_id': 21573, 'p': 0.0616}, {'token': ' k', 'token_id': 413, 'p': 0.0475}, {'token': ' tennis', 'token_id': 22556, 'p': 0.0258}]}]
generated_tokens =  [[[{'token': ' basketball', 'id': 20305, 'p': 0.472900390625}, {'token': ' golf', 'id': 29416, 'p': 0.135498046875}, {'token': ' baseball', 'id': 21573, 'p': 0.061553955078125}, {'token': ' k', 'id': 413, 'p': 0.04754638671875}, {'token': ' tennis', 'id': 22556, 'p': 0.025848388671875}], [{'token': ' .', 'id': 29889, 'p': 0.1436767578125}, {'token': ' like', 'id': 763, 'p': 0.0987548828125}, {'token': ' with', 'id': 